In [3]:
# =========================
# HYDRA-256 PRNG
# =========================

class HYDRA256:
    """
    HYDRA-256 Pseudo-Random Number Generator
    State size: 256 bits
    Counter: 64 bits
    Output: 64 bits
    """

    MASK = (1 << 64) - 1

    C1 = 0x9E3779B97F4A7C15
    C2 = 0xBF58476D1CE4E5B9
    C3 = 0x94D049BB133111EB

    def __init__(self, seed: int):
        if seed == 0:
            raise ValueError("Seed must be non-zero")

        self.s0 = self._smix(seed)
        self.s1 = self._smix(self.s0 ^ self.C1)
        self.s2 = self._smix(self.s1 ^ self.C2)
        self.s3 = self._smix(self.s2 ^ self.C3)

        self.ctr = 1

    @staticmethod
    def rotl(x, k):
        return ((x << k) | (x >> (64 - k))) & HYDRA256.MASK

    def _smix(self, x):
        x &= self.MASK
        x ^= self.rotl(x, 7)
        x = (x * self.C1) & self.MASK
        x ^= self.rotl(x, 41)
        x = (x * self.C2) & self.MASK
        return x

    def _round(self):
        a, b, c, d = self.s0, self.s1, self.s2, self.s3

        a = (a + b + self.ctr) & self.MASK
        b ^= self.rotl(c, 17)
        c = (c + d) & self.MASK
        d ^= self.rotl(a, 45)

        self.s0 = self._smix(a)
        self.s1 = self._smix(b)
        self.s2 = self._smix(c)
        self.s3 = self._smix(d)

    def next_uint64(self):
        self.ctr = (self.ctr + 1) & self.MASK

        self._round()
        self._round()
        self._round()

        out = (
            self.rotl(self.s0, 13) ^
            self.rotl(self.s1, 29) ^
            self.rotl(self.s2, 47) ^
            self.s3
        )

        return self._smix(out)

    def random(self):
        return self.next_uint64() / float(1 << 64)

    def randint(self, low, high):
        return low + self.next_uint64() % (high - low + 1)

    def randbytes(self, n):
        output = bytearray()
        while len(output) < n:
            output.extend(self.next_uint64().to_bytes(8, "little"))
        return bytes(output[:n])


# =========================
# TEST / DEMO (THIS PART WAS MISSING)
# =========================

if __name__ == "__main__":

    rng = HYDRA256(seed=123456789)

    print("=== 64-bit Integers ===")
    for i in range(5):
        print(rng.next_uint64())

    print("\n=== Uniform Floats ===")
    for i in range(3):
        print(rng.random())

    print("\n=== Random Integers [10, 100] ===")
    for i in range(5):
        print(rng.randint(10, 100))

    print("\n=== Random Bytes (32 bytes) ===")
    print(rng.randbytes(32))


=== 64-bit Integers ===
4247409550028494637
8063482995160299695
5657801371382499641
2083685587833636297
1918804618702264272

=== Uniform Floats ===
0.3811731852929238
0.6373368850165403
0.485564874955366

=== Random Integers [10, 100] ===
95
91
62
46
98

=== Random Bytes (32 bytes) ===
b'AKJ\x8d\xdd\xb9\xd5\xac\x07M;\x0f\xf2\x1f\xaf;J=\xc4#x\t\n\xa1h\x8ej\xc3/?R\xd9'


LFSR generation

Berlekamp–Massey cracking (SUCCESS)

HYDRA-256 PRNG

Berlekamp–Massey attack on HYDRA (FAILURE → high linear complexity)

In [4]:
# ============================================================
# PSEUDO-RANDOM NUMBER GENERATORS & CRYPTANALYSIS DEMO
# ============================================================

import math

# ============================================================
# 1. LFSR IMPLEMENTATION
# ============================================================

def createLFSRgenerator(taps, seed):
    def lfsrGen():
        deg = len(taps)
        period = (1 << deg) - 1
        value = seed
        for _ in range(period):
            bit = 0
            for j in range(deg):
                if taps[j]:
                    bit ^= (value >> j)
            bit &= 1
            value = (value >> 1) | (bit << (deg - 1))
            yield value
    return lfsrGen


def getBitFromInt(val, pos):
    return (val >> pos) & 1


# ============================================================
# 2. BERLEKAMP–MASSEY ALGORITHM
# ============================================================

def BerlekampMasseyAlgorithm(sequence):

    def discrepancy(seq, poly, i, L):
        return sum(seq[i - j] & poly[j] for j in range(L + 1)) % 2

    def addPoly(p1, p2, n):
        return [p1[i] ^ p2[i] for i in range(n)]

    N = len(sequence)
    F = [0] * N
    f = [0] * N
    F[0] = f[0] = 1
    L = 0
    delta = 1

    for n in range(N):
        d = discrepancy(sequence, F, n, L)
        if d:
            g = F.copy()
            F = addPoly(F, [0]*delta + f, N)
            if 2 * L <= n:
                L = n + 1 - L
                f = g
                delta = 1
            else:
                delta += 1
        else:
            delta += 1

    return F[:L+1]


def polyToString(poly):
    L = len(poly) - 1
    terms = []
    for i in range(L):
        if poly[i]:
            terms.append(f"x^{L-i}")
    if poly[L]:
        terms.append("1")
    return " + ".join(terms)


def getTapsSequenceFromAnnihilatingPoly(poly):
    p = poly.copy()
    p.reverse()
    return tuple(p[:-1])


# ============================================================
# 3. HYDRA-256 NONLINEAR PRNG
# ============================================================

class HYDRA256:

    MASK = (1 << 64) - 1

    C1 = 0x9E3779B97F4A7C15
    C2 = 0xBF58476D1CE4E5B9
    C3 = 0x94D049BB133111EB

    def __init__(self, seed):
        if seed == 0:
            raise ValueError("Seed must be non-zero")

        self.s0 = self._smix(seed)
        self.s1 = self._smix(self.s0 ^ self.C1)
        self.s2 = self._smix(self.s1 ^ self.C2)
        self.s3 = self._smix(self.s2 ^ self.C3)
        self.ctr = 1

    @staticmethod
    def rotl(x, k):
        return ((x << k) | (x >> (64 - k))) & HYDRA256.MASK

    def _smix(self, x):
        x &= self.MASK
        x ^= self.rotl(x, 7)
        x = (x * self.C1) & self.MASK
        x ^= self.rotl(x, 41)
        x = (x * self.C2) & self.MASK
        return x

    def _round(self):
        a, b, c, d = self.s0, self.s1, self.s2, self.s3

        a = (a + b + self.ctr) & self.MASK
        b ^= self.rotl(c, 17)
        c = (c + d) & self.MASK
        d ^= self.rotl(a, 45)

        self.s0 = self._smix(a)
        self.s1 = self._smix(b)
        self.s2 = self._smix(c)
        self.s3 = self._smix(d)

    def next_uint64(self):
        self.ctr = (self.ctr + 1) & self.MASK
        self._round()
        self._round()
        self._round()
        out = (
            self.rotl(self.s0, 13) ^
            self.rotl(self.s1, 29) ^
            self.rotl(self.s2, 47) ^
            self.s3
        )
        return self._smix(out)


# ============================================================
# 4. ATTACK DEMONSTRATION
# ============================================================

if __name__ == "__main__":

    print("===== LFSR → Berlekamp–Massey (SUCCESS) =====")

    lfsr = createLFSRgenerator((1,0,0,1), 13)()
    lfsr_vals = [next(lfsr) for _ in range(15)]
    lfsr_bits = [getBitFromInt(v, 0) for v in lfsr_vals]

    poly = BerlekampMasseyAlgorithm(lfsr_bits)
    print("Recovered polynomial:", polyToString(poly))
    print("Recovered taps:", getTapsSequenceFromAnnihilatingPoly(poly))

    print("\n===== HYDRA-256 → Berlekamp–Massey (FAILURE) =====")

    hydra = HYDRA256(123456789)
    hydra_bits = [(hydra.next_uint64() & 1) for _ in range(256)]

    hydra_poly = BerlekampMasseyAlgorithm(hydra_bits)
    print("Recovered polynomial degree:", len(hydra_poly) - 1)
    print("Polynomial (meaningless):", polyToString(hydra_poly))

    print("\nInterpretation:")
    print("• LFSR: short linear recurrence → cracked")
    print("• HYDRA-256: no short linear recurrence → BM fails")


===== LFSR → Berlekamp–Massey (SUCCESS) =====
Recovered polynomial: x^4 + x^3 + 1
Recovered taps: (1, 0, 0, 1)

===== HYDRA-256 → Berlekamp–Massey (FAILURE) =====
Recovered polynomial degree: 128
Polynomial (meaningless): x^128 + x^126 + x^123 + x^122 + x^119 + x^118 + x^116 + x^115 + x^113 + x^111 + x^110 + x^109 + x^107 + x^106 + x^103 + x^102 + x^101 + x^100 + x^99 + x^94 + x^93 + x^91 + x^87 + x^86 + x^85 + x^82 + x^81 + x^80 + x^79 + x^78 + x^77 + x^74 + x^72 + x^68 + x^67 + x^65 + x^63 + x^62 + x^61 + x^60 + x^59 + x^57 + x^56 + x^53 + x^44 + x^41 + x^40 + x^39 + x^37 + x^36 + x^35 + x^34 + x^32 + x^31 + x^30 + x^27 + x^26 + x^21 + x^16 + x^15 + x^14 + x^13 + x^11 + x^9 + x^8 + x^7 + x^6 + x^3 + 1

Interpretation:
• LFSR: short linear recurrence → cracked
• HYDRA-256: no short linear recurrence → BM fails
